# Face Mask Detection Model Training
This notebook trains a CNN model to detect face masks

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from utils.helpers import preprocess_face

print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")

In [ ]:
# Create synthetic training data (since we don't have a real dataset)
def create_synthetic_data(num_samples=1000):
    """
    Create synthetic face mask data for training
    In real scenario, you would load actual face images with/without masks
    """
    X = []
    y = []
    
    for i in range(num_samples):
        # Create random face-like image
        img = np.random.randint(50, 200, (150, 150, 3), dtype=np.uint8)
        
        # Add some face-like features
        # Eyes
        cv2.circle(img, (50, 50), 10, (0, 0, 0), -1)
        cv2.circle(img, (100, 50), 10, (0, 0, 0), -1)
        
        # Nose
        cv2.circle(img, (75, 75), 5, (150, 100, 100), -1)
        
        # Decide if this face has a mask (50% probability)
        has_mask = np.random.choice([0, 1])
        
        if has_mask:
            # Add mask-like rectangle in lower face area
            mask_color = np.random.choice([
                [255, 255, 255],  # White mask
                [100, 150, 255],  # Blue mask
                [50, 50, 50],     # Black mask
            ])
            cv2.rectangle(img, (30, 90), (120, 130), mask_color, -1)
            
            # Add some mask texture
            for _ in range(10):
                x = np.random.randint(30, 120)
                y = np.random.randint(90, 130)
                cv2.circle(img, (x, y), 2, (mask_color[0]//2, mask_color[1]//2, mask_color[2]//2), -1)
        
        # Normalize and add to dataset
        img_normalized = img / 255.0
        X.append(img_normalized)
        y.append(has_mask)
    
    return np.array(X), np.array(y)

# Generate synthetic data
print("Creating synthetic training data...")
X, y = create_synthetic_data(2000)
print(f"Created {len(X)} samples")
print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Mask samples: {np.sum(y)}, No mask samples: {len(y) - np.sum(y)}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Training Data')

for i in range(10):
    row = i // 5
    col = i % 5
    
    axes[row, col].imshow(X[i])
    axes[row, col].set_title(f"{'Mask' if y[i] == 1 else 'No Mask'}")
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Build the CNN model
def create_mask_detection_model():
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        # Fourth Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        # Flatten and Dense layers
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model

# Create and compile the model
model = create_mask_detection_model()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

In [ ]:
# Train the model
print("Training the model...")

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1
)

print("Training completed!")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Make predictions on test set
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Calculate metrics
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("\nClassification Report:")
print(classification_report(y_test, predicted_classes, target_names=['No Mask', 'Mask']))

# Confusion Matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Mask', 'Mask'], 
            yticklabels=['No Mask', 'Mask'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Save the trained model
model.save('model/face_mask_model.h5')
print("Model saved successfully as 'model/face_mask_model.h5'")

# Also save in SavedModel format for deployment
model.save('model/face_mask_model_savedmodel')
print("Model also saved in SavedModel format for deployment")

In [ ]:
# Test the saved model
print("Testing the saved model...")
loaded_model = tf.keras.models.load_model('model/face_mask_model.h5')

# Test on a few samples
test_samples = X_test[:5]
test_labels = y_test[:5]

predictions = loaded_model.predict(test_samples)

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    axes[i].imshow(test_samples[i])
    pred_prob = predictions[i][0]
    pred_label = 'Mask' if pred_prob > 0.5 else 'No Mask'
    true_label = 'Mask' if test_labels[i] == 1 else 'No Mask'
    
    axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {pred_prob:.2f}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("Model testing completed successfully!")